<a href="https://colab.research.google.com/github/alvoera2/DataScience/blob/main/gender_clasification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
print('done')

done


**Context**

While I was practicing machine learning, I wanted to create a simple dataset that is closely aligned to the real world scenario and gives better results to whet my appetite on this domain. If you are a beginner who wants to try solving classification problems in machine learning and if you prefer achieving better results, try using this dataset in your projects which will be a great place to start.

**Content**

This dataset contains 7 features and a label column.



*   long_hair - This column contains 0's and 1's where 1 is "long hair" and 0 is "not long hair".

*  forehead_width_cm - This column is in CM's. This is the width of the forehead.
*  forehead_height_cm - This is the height of the forehead and it's in Cm's.
*  nose_wide - This column contains 0's and 1's where 1 is "wide nose" and 0 is "not wide nose".
*  nose_long - This column contains 0's and 1's where 1 is "Long nose" and 0 is "not long nose".
*  lips_thin - This column contains 0's and 1's where 1 represents the "thin lips" while 0 is "Not thin lips".
*  distance_nose_to_lip_long - This column contains 0's and 1's where 1 represents the "long distance between nose and lips" while 0 is "short distance between nose and lips".

*  gender - This is either "Male" or "Female".



**SIAPKAN DATA TANPA FILLNA DAN GET DUMMIES**

In [94]:
df = pd.read_csv('/content/drive/MyDrive/DATASET/gender_classification_v7.csv')
df.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,1,11.8,6.1,1,0,1,1,Male
1,0,14.0,5.4,0,0,1,0,Female
2,0,11.8,6.3,1,1,1,1,Male
3,0,14.4,6.1,0,1,1,1,Male
4,1,13.5,5.9,0,0,0,0,Female


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   long_hair                  5001 non-null   int64  
 1   forehead_width_cm          5001 non-null   float64
 2   forehead_height_cm         5001 non-null   float64
 3   nose_wide                  5001 non-null   int64  
 4   nose_long                  5001 non-null   int64  
 5   lips_thin                  5001 non-null   int64  
 6   distance_nose_to_lip_long  5001 non-null   int64  
 7   gender                     5001 non-null   object 
dtypes: float64(2), int64(5), object(1)
memory usage: 312.7+ KB


In [96]:
df.isna().sum()

long_hair                    0
forehead_width_cm            0
forehead_height_cm           0
nose_wide                    0
nose_long                    0
lips_thin                    0
distance_nose_to_lip_long    0
gender                       0
dtype: int64

kenapa split data diawal? untuk menghindari leakage

In [97]:
X = df.drop(columns='gender')
y = df['gender']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3500, 7), (1501, 7), (3500,), (1501,))

**PREPOCESSOR**

setelah melakukan splitting data baru kita akan melakukan preprocessing data seperti fillna dan get dummies dan lainnya

In [98]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder

In [99]:
numerical_pipeline = Pipeline([
    ("imputer",SimpleImputer(strategy="mean")),
    ("scaler",MinMaxScaler())
])

categorical_pipeline = Pipeline([
    ("imputer",SimpleImputer(strategy="most_frequent")),
    ("onehot",OneHotEncoder())
])

In [100]:
X.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long
0,1,11.8,6.1,1,0,1,1
1,0,14.0,5.4,0,0,1,0
2,0,11.8,6.3,1,1,1,1
3,0,14.4,6.1,0,1,1,1
4,1,13.5,5.9,0,0,0,0


In [101]:
from sklearn.compose import ColumnTransformer

In [102]:
preprocessor = ColumnTransformer([
    ("numeric",numerical_pipeline,['forehead_width_cm','forehead_height_cm']),
    ("categoric",categorical_pipeline,['long_hair','nose_wide','nose_long','lips_thin','distance_nose_to_lip_long'])
])

In [103]:
from sklearn.neighbors import KNeighborsClassifier


In [104]:
pipeline = Pipeline([
    ("prep",preprocessor),
    ("algo",KNeighborsClassifier())
])

**MODEL**

In [105]:
from sklearn.model_selection import GridSearchCV

In [106]:
parameter= {
    'algo__n_neighbors': range(1,51,2),
    'algo__weights': ['uniform','distance'],
    'algo__p': [1,2],
    'algo__algorithm':['ball_tree','kd_tree','brute']

}

model = GridSearchCV(pipeline,parameter, cv=5,verbose =1)

In [107]:
model.fit(X_train,y_train)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['forehead_width_cm',
                                                                          'forehead_height_cm']),
                                                                        ('categoric',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder())]),
                                                                         ['long_hair',
                                                                          'nose_wide',
                                                                          'nose_long',
                                                                          'lips_thin',
                                                                          'distance_nose_to_lip_long'])])),
                                       ('algo', KNeighborsClassifier())]),
             param_grid={'algo__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'algo__n_neighbors': range(1, 51, 2),
                         'algo__p': [1, 2],
                         'algo__weights': ['uniform', 'distance']},
             verbose=1)

In [ ]:
pd.DataFrame(model.cv_results_).sort_values(by="rank_test_score")

In [109]:
model.best_params_

{'algo__algorithm': 'ball_tree',
 'algo__n_neighbors': 49,
 'algo__p': 2,
 'algo__weights': 'uniform'}

In [110]:
model.score(X_train,y_train), model.score(X_test,y_test)

(0.9731428571428572, 0.960692871419054)